In [2]:
import pandas as pd
import numpy as np
from config import *
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix


from pgmpy.models import BayesianNetwork, BayesianModel
from pgmpy.estimators import HillClimbSearch, MaximumLikelihoodEstimator, BicScore, BayesianEstimator, PC
from pgmpy.inference import DBNInference, VariableElimination, BeliefPropagation

In [3]:
# read csv
df = pd.read_csv(german_risk, index_col=None)

In [4]:
df.isnull().sum()

Unnamed: 0            0
Age                   0
Sex                   0
Job                   0
Housing               0
Saving accounts     183
Checking account    394
Credit amount         0
Duration              0
Purpose               0
Risk                  0
dtype: int64

In [5]:
#Looking unique values
print(df.nunique())
#Looking the data
print(df.head())

Unnamed: 0          1000
Age                   53
Sex                    2
Job                    4
Housing                3
Saving accounts        4
Checking account       3
Credit amount        921
Duration              33
Purpose                8
Risk                   2
dtype: int64
   Unnamed: 0  Age     Sex  ...  Duration              Purpose  Risk
0           0   67    male  ...         6             radio/TV  good
1           1   22  female  ...        48             radio/TV   bad
2           2   49    male  ...        12            education  good
3           3   45    male  ...        42  furniture/equipment  good
4           4   53    male  ...        24                  car   bad

[5 rows x 11 columns]


In [6]:
df["Credit amount"].max(), df["Credit amount"].min()

(18424, 250)

In [7]:
data = df.fillna('unknown')
data["Risk"] = data['Risk'].replace({"good": 0, "bad": 1})

/tmp/ipykernel_75280/4242558525.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data["Risk"] = data['Risk'].replace({"good": 0, "bad": 1})


In [8]:
# for col in ["Age", "Job" ,"Credit amount", "Duration"]:
interval = (18, 25, 35, 60, 120)
cats = ['Student', 'Young', 'Adult', 'Senior']
data["Age"] = pd.cut(data["Age"], interval, labels=cats) 
data["Credit amount"] = pd.qcut(data["Credit amount"], q=5, labels = ["very low", "low", "mid", "high", "very high"]) 
data


,Unnamed: 0,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,0,Senior,male,2,own,unknown,little,very low,6,radio/TV,0
1,1,Student,female,2,own,little,moderate,very high,48,radio/TV,1
2,2,Adult,male,1,own,little,unknown,mid,12,education,0
3,3,Adult,male,2,free,little,little,very high,42,furniture/equipment,0
4,4,Adult,male,2,free,little,little,very high,24,car,1
...,...,...,...,...,...,...,...,...,...,...,...
995,995,Young,female,1,own,little,unknown,low,12,furniture/equipment,0
996,996,Adult,male,3,own,little,little,high,30,car,0
997,997,Adult,male,2,own,little,unknown,very low,12,radio/TV,0
998,998,Student,male,2,free,little,little,low,45,radio/TV,1


In [9]:
X_train, X_test, y_train, y_test = train_test_split(data, data["Risk"], test_size=0.1)

In [12]:
hc = PC(X_train)
best_model = hc.estimate(scoring_method=BicScore(X_train))
edges = list(best_model.edges())
model = BayesianNetwork(edges)

Working for n conditional variables: 0:   0%|          | 0/5 [00:00<?, ?it/s]/home/jiqing/miniconda3/envs/cs5340/lib/python3.12/site-packages/pgmpy/estimators/CITests.py:539: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  data.groupby([X, Y]).size().unstack(Y, fill_value=0), lambda_=lambda_
/home/jiqing/miniconda3/envs/cs5340/lib/python3.12/site-packages/pgmpy/estimators/CITests.py:539: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  data.groupby([X, Y]).size().unstack(Y, fill_value=0), lambda_=lambda_
/home/jiqing/miniconda3/envs/cs5340/lib/python3.12/site-packages/pgmpy/estimators/CITests.py:539: FutureWarning

In [13]:
print(edges)

[('Purpose', 'Job'), ('Duration', 'Credit amount'), ('Housing', 'Age'), ('Sex', 'Age'), ('Sex', 'Housing'), ('Checking account', 'Saving accounts'), ('Risk', 'Checking account')]


In [38]:
test_dict = X_test[["Age", 'Checking account', 'Saving accounts',"Credit amount", "Housing", "Sex"]].to_dict("records")

In [26]:
# model = BayesianNetwork([('Age', 'Job'), ("Sex", 'Job'), ('Job', 'Housing'), 
#                          ("Age", "Risk"), ("Saving accounts", "Risk"), ("Sex", "Risk"),
#                          ('Credit amount', 'Risk'), ('Housing', 'Risk')])
model.fit(X_train, estimator=BayesianEstimator)

NameError: name 'BayesianEstimator' is not defined

In [27]:
infer = VariableElimination(model)
result = []
for i in test_dict:
    query_result = infer.query(variables=['Risk'], evidence=i)
    result.append(np.argmax(query_result.values))
confusion_matrix(y_test, result, labels=[0, 1])

ValueError: No CPD associated with Age

In [ ]:
from pgmpy.factors.discrete.CPD import TabularCPD

def print_full(cpd):
    backup = TabularCPD._truncate_strtable
    TabularCPD._truncate_strtable = lambda self, x: x
    print(cpd)
    TabularCPD._truncate_strtable = backup

In [ ]:
print_full(model.get_cpds('Risk'))

+------------------+--------------------------+----------------------------+------------------------+---------------------------+
| Checking account | Checking account(little) | Checking account(moderate) | Checking account(rich) | Checking account(unknown) |
+------------------+--------------------------+----------------------------+------------------------+---------------------------+
| Risk(0)          | 0.4959016393442623       | 0.6056910569105691         | 0.7758620689655172     | 0.8721590909090909        |
+------------------+--------------------------+----------------------------+------------------------+---------------------------+
| Risk(1)          | 0.5040983606557377       | 0.3943089430894309         | 0.22413793103448276    | 0.1278409090909091        |
+------------------+--------------------------+----------------------------+------------------------+---------------------------+
